In [1]:
from keras.backend.tensorflow_backend import set_session
from Models import ModelMulti
import tensorflow as tf

session_config = tf.ConfigProto()
session_config.gpu_options.visible_device_list = "0"
session_config.gpu_options.allow_growth = True
set_session(tf.Session(config=session_config))

model_1 = Model_Multi()
model_1.train_predict()


/home/aamomeni/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


IndentationError: expected an indented block (Models.py, line 153)

**Datasets**

In [ ]:
import pandas as pd 
import os 
import numpy as np 
import matplotlib.pyplot as plt 
from PIL import Image
import cv2

In [ ]:
#Concatenate LQ, MQ, HQ

suffix = ".csv"
csv_directory = "Datasets/"
csv_files = [i for i in os.listdir(csv_directory) if i.endswith( suffix )]

full_data = []
for i in range(len(csv_files)):
    data = pd.read_csv(csv_directory+'/'+csv_files[i], sep=';', index_col = 0)
    full_data.append(data)

full_data = pd.concat(full_data, axis=0)
print(full_data.shape)
full_data.head(25)



full_data =  pd.read_csv('Datasets/HQ_digital.csv', sep=';', index_col = 0)
full_data = full_data.replace("X", 10)

full_data.head()

In [ ]:
#Only use data for which we have images

frame_directory = "Datasets_frames/"

data = full_data[full_data["image"].isin(os.listdir(frame_directory))]
print(data.shape)
data.head(40)

**Split into test and validation sets**

In [ ]:
from sklearn.cross_validation import train_test_split

X = data.iloc[:,1]
y = data.iloc[:,2:]

ids_train, ids_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=1)

In [ ]:
y_train_vect = [y_train["cadran_1"], y_train["cadran_2"], y_train["cadran_3"], y_train["cadran_4"]]
y_val_vect =  [y_val["cadran_1"], y_val["cadran_2"], y_val["cadran_3"], y_val["cadran_4"]]

**Convert to arrays**

In [ ]:
def convert_to_arrays(samples, directory):
    X = []
    for sample in samples:
        ID =  directory + "%s" % (sample)
        img = Image.open(ID)
        img = np.array(img)
        img = cv2.equalizeHist(img)
        img = cv2.threshold(img, 40, 255, cv2.THRESH_BINARY_INV)[1]
        img = cv2.dilate(img, None, iterations=1)
        img = img[:,:245]
        img = img.reshape((img.shape[0],img.shape[1],1))
        X.append(img)
    X = np.asarray(X)
    return X

X_train = convert_to_arrays(ids_train, frame_directory)
X_val = convert_to_arrays(ids_val, frame_directory)

**CNN Model**

In [ ]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers.core import Dropout, Activation
from keras.layers import BatchNormalization
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
from keras.optimizers import Adam, SGD
from keras import regularizers
import keras.backend
import tensorflow as tf


session_config = tf.ConfigProto()
session_config.gpu_options.visible_device_list = "0"
session_config.gpu_options.allow_growth = True
set_session(tf.Session(config=session_config))


model_input = Input((100,245,1))

x = Conv2D(32, (3, 3), padding='same', name='conv2d_hidden_1', kernel_regularizer=regularizers.l2(0.01))(model_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(3, 3),name='maxpool_2d_hidden_1')(x)
x = Dropout(0.30)(x)

x = Conv2D(64, (3, 3), padding='same', name='conv2d_hidden_2', kernel_regularizer=regularizers.l2(0.01))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(3, 3),name='maxpool_2d_hidden_2')(x)
x = Dropout(0.30)(x)

x = Conv2D(128, (3, 3), padding='same', name='conv2d_hidden_3', kernel_regularizer=regularizers.l2(0.01))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(3, 3),name='maxpool_2d_hidden_3')(x)
x = Dropout(0.30)(x)

x = Flatten()(x)

x = Dense(256, activation ='relu', kernel_regularizer=regularizers.l2(0.01))(x)

digit1 = (Dense(output_dim =11,activation = 'softmax', name='digit_1'))(x)
digit2 = (Dense(output_dim =11,activation = 'softmax', name='digit_2'))(x)
digit3 = (Dense(output_dim =11,activation = 'softmax', name='digit_3'))(x)
digit4 = (Dense(output_dim =11,activation = 'softmax', name='digit_4'))(x)

outputs = [digit1, digit2, digit3, digit4]

M = Model(input = model_input , output = outputs)
M._make_predict_function()

opt = Adam(lr=0.001, decay=0.0001)

M.compile(loss="sparse_categorical_crossentropy", optimizer= opt, metrics = ['accuracy'])

keras.backend.get_session().run(tf.initialize_all_variables())

history =M.fit(X_train, y_train_vect, batch_size= 50, nb_epoch=10, verbose=1, validation_data=(X_val, y_val_vect))

In [ ]:
# Acc Curves

for i in range(1,5):
        plt.figure(figsize=[8,6])
        plt.plot(history.history['digit_%i_acc' %i],'r',linewidth=0.5)
        plt.plot(history.history['val_digit_%i_acc' %i],'b',linewidth=0.5)
        plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
        plt.xlabel('Epochs ',fontsize=16)
        plt.ylabel('Accuracy',fontsize=16)
        plt.title('Accuracy Curves Digit %i' %i,fontsize=16)
        plt.show()


In [ ]:
# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(history.history['digit_2_loss'],'r',linewidth=0.5)
plt.plot(history.history['val_digit_2_loss'],'b',linewidth=0.5)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
plt.show()

In [ ]:
ID = 'Datasets_frames/08d21828339e1a6adcd892ef2f0d0866083c51bf.jpg'
img = Image.open(ID)
img = np.array(img)
img = cv2.equalizeHist(img)
img = cv2.threshold(img, 40, 255, cv2.THRESH_BINARY_INV)[1]
img = cv2.dilate(img, None, iterations=1)
img = img[:,:245]
plt.imshow(img, cmap=plt.cm.gray)
plt.show()


In [ ]:
img = img.reshape((1,img.shape[0],img.shape[1],1))
preds = M.predict(img)

In [ ]:
for i in range(len(preds)):
    print(np.argmax(preds[i]))

In [ ]:
y_pred = M.predict(X_val)

In [ ]:
correct_preds = 0

for i in range(X_val.shape[0]):         
    pred_list_i = [np.argmax(pred[i]) for pred in y_pred]
    val_list_i  = y_val.values[i].astype('int')
    if np.array_equal(val_list_i, pred_list_i):
        correct_preds = correct_preds + 1

        
mse = 0 
diff = []


for i in range(X_val.shape[0]):
        pred_list_i = [np.argmax(pred[i]) for pred in y_pred]
        pred_number = 1000* pred_list_i[0] + 100* pred_list_i[1] + 10 * pred_list_i[2] + 1* pred_list_i[3]
        val_list_i  = y_val.values[i].astype('int')
        val_number = 1000* val_list_i[0] + 100*  val_list_i[1] + 10 *  val_list_i[2] + 1*  val_list_i[3]
        diff.append(val_number - pred_number)
        

np.sort(diff)
        
        
    

In [ ]:
val_list_i  = [val[i] for val in y_val]
    matching_preds = [pred.argmax(-1) == val.argmax(-1) for pred, val in zip(pred_list_i, val_list_i)]
    correct_preds = int(np.all(matching_preds))

total_acc = (correct_preds / float(X_test.shape[0]))*100
print(total_acc)

**MNIST Model**

In [ ]:
ids = []
labels = []
for i in range(11):
    directory = 'Datasets_digits/%i/' %i
    for j in os.listdir(directory):
        if j != '.DS_S':
            ids.append(directory+j)
            labels.append(i)

data = pd.DataFrame(list(zip(ids,labels)))

data.head()

In [ ]:
ids = []
labels = []
for i in range(10):
    directory = 'Datasets_Eleven/%i/' %i
    for j in os.listdir(directory):
        ids.append(directory+j)
        labels.append(i)
data = pd.DataFrame(list(zip(ids,labels)))
data.head()

In [ ]:
X = data.iloc[:,0]
y = data.iloc[:,1]

ids_train, ids_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=1)

def convert_digits_to_arrays(samples):
    X = []
    for sample in samples:
        img = Image.open(sample)
        img = np.array(img)
        #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        img.resize((30,50))
        img = cv2.equalizeHist(img)
        img = cv2.threshold(img, 40, 255, cv2.THRESH_BINARY_INV)[1]
        img = cv2.dilate(img, None, iterations=1)
        img = np.array(img)
        img = img.reshape((30,50,1))
        X.append(img)
    X = np.asarray(X)
    return X

X_train = convert_digits_to_arrays(ids_train)
X_val = convert_digits_to_arrays(ids_val)

In [ ]:
model_input = Input((30,50,1))

x = Conv2D(32, (3, 3), padding='same', name='conv2d_hidden_1', kernel_regularizer=regularizers.l2(0.01))(model_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(3, 3),name='maxpool_2d_hidden_1')(x)
x = Dropout(0.30)(x)

x = Conv2D(63, (3, 3), padding='same', name='conv2d_hidden_2', kernel_regularizer=regularizers.l2(0.01))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(3, 3),name='maxpool_2d_hidden_2')(x)
x = Dropout(0.30)(x)

x = Conv2D(128, (3, 3), padding='same', name='conv2d_hidden_3', kernel_regularizer=regularizers.l2(0.01))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2), strides=(3, 3),name='maxpool_2d_hidden_3')(x)
x = Dropout(0.30)(x)

x = Flatten()(x)

x = Dense(1024, activation ='relu', kernel_regularizer=regularizers.l2(0.01))(x)

output = Dense(output_dim =11,activation = 'softmax', name='output')(x)

M = Model(input = model_input , output = output)

opt = Adam(lr=0.001)

M.compile(loss="sparse_categorical_crossentropy", optimizer= opt, metrics = ['accuracy'])

history = M.fit(X_train, y_train, batch_size= 32, nb_epoch=30, verbose=1, validation_data=(X_val, y_val))

In [ ]:
y_pred = M.predict(X_val)

e = []
for i in range(X_val.shape[0]):
    val_id = ids_val.values[i]
    pred_list_i = np.argmax(y_pred[i]).astype('int')
    val_list_i  = y_val.values[i].astype('int')
    if val_list_i != pred_list_i:
        e.append(val_id.split('/')[2].split('-')[0])
np.unique(e)
#        print(val_id, pred_list_i, val_list_i)

In [ ]:
val_id.split('/')[2].split('-')[0]